In [1]:
%%capture
!pip install mne
import os
from glob import glob
import numpy as np
import pandas as pd
import mne
import warnings
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
import random
random.seed(42)
np.random.seed(42)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data=r'/content/drive/MyDrive/eeg-data-row/session1_sub17_reaching_realMove.vhdr'
file_paths=[data]

In [4]:
#'/content/drive/MyDrive/eeg-raw-data-MI/session1_sub12_reaching_MI.vhdr'

In [5]:
channel_names = [
    'Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8',  # Frontal Channels
    'C3', 'C4', 'Cz',  # Central Channels
    'P3', 'P4', 'P7', 'P8',  # Parietal Channels
    'O1', 'O2',  # Occipital Channels
    'T3', 'T4', 'T5', 'T6',  # Temporal Channels
    'Fz', 'FCz', 'CPz', 'FT7', 'FT8', 'TP7', 'TP8', 'Oz'  # Additional Channels
]

In [6]:
def read_data(path):
    data=path
    raw = mne.io.read_raw_brainvision(data, preload=True,eog=['hEOG_L','hEOG_R','vEOG_U','vEOG_D'])
    raw.drop_channels(['hEOG_L','hEOG_R','vEOG_U','vEOG_D','EMG_1','EMG_2','EMG_3','EMG_4','EMG_5','EMG_6','EMG_ref'])
    #raw.drop_channels(['eog-r', 'eog-m', 'eog-l'])
    raw.pick_channels(channel_names)
    raw.pick_types(eog=False , emg = False , eeg= True )
    raw.set_eeg_reference()
    #raw.resample(sfreq=1250)
    raw.filter(l_freq=0.5,h_freq=30)
    events=mne.events_from_annotations(raw)
    epoch=mne.Epochs(raw,events[0],event_id=[11,21,31,41,51,61],tmin=0,tmax=2, baseline=(0, 0))
    labels=epoch.events[:,-1]
    features=epoch.get_data()
    return features , labels

In [7]:
def reshape_data (data_epochs):
  # add depth 1 to the data
  data_epochs= np.reshape(data_epochs,(data_epochs.shape[0],data_epochs.shape[1],data_epochs.shape[2],1))
  print (data_epochs.shape)
  final_array=np.vstack(data_array)
  print(data_epochs.shape)
  '''data_epochs=np.moveaxis(data_epochs,1,2)
  #data_epochs.shape'''
  return data_epochs

# so lets read all data


In [8]:
#selected=['AF7','AF3','AFz','F7','F5','F3','F1','Fz','FC5','FC3','FC1','FCz','T7','C5','C3','C1','Cz']
data_array , label_array =read_data( file_paths[0])
# label_array=[read_data_label(i) for i in file_paths]

Extracting parameters from /content/drive/MyDrive/eeg-data-row/session1_sub1_reaching_realMove.vhdr...
Setting channel info structure...
Reading 0 ... 8434299  =      0.000 ...  3373.720 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:   11.4s finished


Used Annotations descriptions: ['Comment/actiCAP Active Shield On', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6', 'Stimulus/S  8', 'Stimulus/S 11', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 21', 'Stimulus/S 31', 'Stimulus/S 41', 'Stimulus/S 51', 'Stimulus/S 61']
Not setting metadata
300 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 300 events and 5001 original time points ...
0 bad epochs dropped


In [9]:
%%capture
!pip install PyRiemann>=0.2.5
!pip install scikit-learn>=0.20.1
!pip install matplotlib>=2.2.3
!pip install tensorflow==2.3

In [10]:
#reshape data
data_epochs = reshape_data(data_array)
labels=np.hstack(label_array)

#split data
X_train, X_test, y_train, y_test=train_test_split(data_epochs,labels,test_size = 0.1,stratify=labels,shuffle=True)#,stratify=final_label,shuffle=True
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
print(pd.DataFrame(y_test).value_counts())

# map events to 0-6
d={11:0,21:1,31:2,41:3,51:4,61:5,8:6}
df=pd.DataFrame(y_train)
df=df.replace(d)
y_train=tf.keras.utils.to_categorical(df)#
df=pd.DataFrame(y_test)
df=df.replace(d)
y_test=tf.keras.utils.to_categorical(df)#

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train.reshape(-1,X_train.shape[-1])).reshape(X_train.shape)
X_test=scaler.transform(X_test.reshape(-1,X_test.shape[-1])).reshape(X_test.shape)

print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(300, 22, 5001, 1)
(300, 22, 5001, 1)
(270, 22, 5001, 1) (30, 22, 5001, 1) (270,) (30,)
11    5
21    5
31    5
41    5
51    5
61    5
dtype: int64
(270, 22, 5001, 1) (30, 22, 5001, 1) (270, 6) (30, 6)


In [11]:
"""
 ARL_EEGModels - A collection of Convolutional Neural Network models for EEG
 Signal Processing and Classification, using Keras and Tensorflow
 Requirements:
    (1) tensorflow == 2.X (as of this writing, 2.0 - 2.3 have been verified
        as working)

 To run the EEG/MEG ERP classification sample script, you will also need
    (4) mne >= 0.17.1
    (5) PyRiemann >= 0.2.5
    (6) scikit-learn >= 0.20.1
    (7) matplotlib >= 2.2.3

 To use:

    (1) Place this file in the PYTHONPATH variable in your IDE (i.e.: Spyder)
    (2) Import the model as

        from EEGModels import EEGNet

        model = EEGNet(nb_classes = ..., Chans = ..., Samples = ...)

    (3) Then compile and fit the model

        model.compile(loss = ..., optimizer = ..., metrics = ...)
        fitted    = model.fit(...)
        predicted = model.predict(...)
 Portions of this project are works of the United States Government and are not
 subject to domestic copyright protection under 17 USC Sec. 105.  Those
 portions are released world-wide under the terms of the Creative Commons Zero
 1.0 (CC0) license.

 Other portions of this project are subject to domestic copyright protection
 under 17 USC Sec. 105.  Those portions are licensed under the Apache 2.0
 license.  The complete text of the license governing this material is in
 the file labeled LICENSE.TXT that is a part of this project's official
 distribution.
"""

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K


def EEGNet(nb_classes, Chans = 64, Samples = 128,
             dropoutRate = 0.5, kernLength = 64, F1 = 8,
              D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:

        1. Depthwise Convolutions to learn spatial filters within a
        temporal convolution. The use of the depth_multiplier option maps
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn
        spatial filters within each filter in a filter-bank. This also limits
        the number of free parameters to fit when compared to a fully-connected
        convolution.

        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions.


    While the original paper used Dropout, we found that SpatialDropout2D
    sometimes produced slightly better results for classification of ERP
    signals. However, SpatialDropout2D significantly reduced performance
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.

    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the
    kernel lengths for double the sampling rate, etc). Note that we haven't
    tested the model performance with this rule so this may not work well.

    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
  	advised to do some model searching to get optimal performance on your
  	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D.
    Inputs:

      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D.
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.

    """
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')

    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False,
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)

    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)

    flatten      = Flatten(name = 'flatten')(block2)

    dense        = Dense(nb_classes, name = 'dense',
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)

    return Model(inputs=input1, outputs=softmax)




def EEGNet_SSVEP(nb_classes =12, Chans =8 , Samples =256,
             dropoutRate = 0.5, kernLength = 256, F1 = 96,
             D = 1, F2 = 96, dropoutType = 'Dropout'):
    """ SSVEP Variant of EEGNet, as used in [1].
    Inputs:

      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn.
      D               : number of spatial filters to learn within each temporal
                        convolution.
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.


    [1]. Waytowich, N. et. al. (2018). Compact Convolutional Neural Networks
    for Classification of Asynchronous Steady-State Visual Evoked Potentials.
    Journal of Neural Engineering vol. 15(6).
    http://iopscience.iop.org/article/10.1088/1741-2552/aae5d8
    """

    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')

    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False,
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)

    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)

    flatten      = Flatten(name = 'flatten')(block2)

    dense        = Dense(nb_classes, name = 'dense')(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)

    return Model(inputs=input1, outputs=softmax)



def EEGNet_old(nb_classes, Chans = 64, Samples = 128, regRate = 0.0001,
           dropoutRate = 0.25, kernels = [(2, 32), (8, 4)], strides = (2, 4)):
    """ Keras Implementation of EEGNet_v1 (https://arxiv.org/abs/1611.08024v2)
    This model is the original EEGNet model proposed on arxiv
            https://arxiv.org/abs/1611.08024v2

    with a few modifications: we use striding instead of max-pooling as this
    helped slightly in classification performance while also providing a
    computational speed-up.

    Note that we no longer recommend the use of this architecture, as the new
    version of EEGNet performs much better overall and has nicer properties.

    Inputs:

        nb_classes     : total number of final categories
        Chans, Samples : number of EEG channels and samples, respectively
        regRate        : regularization rate for L1 and L2 regularizations
        dropoutRate    : dropout fraction
        kernels        : the 2nd and 3rd layer kernel dimensions (default is
                         the [2, 32] x [8, 4] configuration)
        strides        : the stride size (note that this replaces the max-pool
                         used in the original paper)

    """

    # start the model
    input_main   = Input((Chans, Samples))
    layer1       = Conv2D(16, (Chans, 1), input_shape=(Chans, Samples, 1),
                                 kernel_regularizer = l1_l2(l1=regRate, l2=regRate))(input_main)
    layer1       = BatchNormalization()(layer1)
    layer1       = Activation('elu')(layer1)
    layer1       = Dropout(dropoutRate)(layer1)

    permute_dims = 2, 1, 3
    permute1     = Permute(permute_dims)(layer1)

    layer2       = Conv2D(4, kernels[0], padding = 'same',
                            kernel_regularizer=l1_l2(l1=0.0, l2=regRate),
                            strides = strides)(permute1)
    layer2       = BatchNormalization()(layer2)
    layer2       = Activation('elu')(layer2)
    layer2       = Dropout(dropoutRate)(layer2)

    layer3       = Conv2D(4, kernels[1], padding = 'same',
                            kernel_regularizer=l1_l2(l1=0.0, l2=regRate),
                            strides = strides)(layer2)
    layer3       = BatchNormalization()(layer3)
    layer3       = Activation('elu')(layer3)
    layer3       = Dropout(dropoutRate)(layer3)

    flatten      = Flatten(name = 'flatten')(layer3)

    dense        = Dense(nb_classes, name = 'dense')(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)

    return Model(inputs=input_main, outputs=softmax)



def DeepConvNet(nb_classes, Chans = 64, Samples = 256,
                dropoutRate = 0.5):
    """ Keras implementation of the Deep Convolutional Network as described in
    Schirrmeister et. al. (2017), Human Brain Mapping.

    This implementation assumes the input is a 2-second EEG signal sampled at
    128Hz, as opposed to signals sampled at 250Hz as described in the original
    paper. We also perform temporal convolutions of length (1, 5) as opposed
    to (1, 10) due to this sampling rate difference.

    Note that we use the max_norm constraint on all convolutional layers, as
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication
    with the original authors.

                      ours        original paper
    pool_size        1, 2        1, 3
    strides          1, 2        1, 3
    conv filters     1, 5        1, 10

    Note that this implementation has not been verified by the original
    authors.

    """

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(25, (1, 5),
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(25, (Chans, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
    block1       = Activation('elu')(block1)
    block1       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block1)
    block1       = Dropout(dropoutRate)(block1)

    block2       = Conv2D(50, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block2       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block2)
    block2       = Activation('elu')(block2)
    block2       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block2)
    block2       = Dropout(dropoutRate)(block2)

    block3       = Conv2D(100, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block2)
    block3       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block3)
    block3       = Activation('elu')(block3)
    block3       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block3)
    block3       = Dropout(dropoutRate)(block3)

    block4       = Conv2D(200, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block3)
    block4       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block4)
    block4       = Activation('elu')(block4)
    block4       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block4)
    block4       = Dropout(dropoutRate)(block4)

    flatten      = Flatten()(block4)

    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)

    return Model(inputs=input_main, outputs=softmax)


# need these for ShallowConvNet
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))


def ShallowConvNet(nb_classes, Chans = 64, Samples = 128, dropoutRate = 0.5):
    """ Keras implementation of the Shallow Convolutional Network as described
    in Schirrmeister et. al. (2017), Human Brain Mapping.

    Assumes the input is a 2-second EEG signal sampled at 128Hz. Note that in
    the original paper, they do temporal convolutions of length 25 for EEG
    data sampled at 250Hz. We instead use length 13 since the sampling rate is
    roughly half of the 250Hz which the paper used. The pool_size and stride
    in later layers is also approximately half of what is used in the paper.

    Note that we use the max_norm constraint on all convolutional layers, as
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication
    with the original authors.

                     ours        original paper
    pool_size        1, 35       1, 75
    strides          1, 7        1, 15
    conv filters     1, 13       1, 25

    Note that this implementation has not been verified by the original
    authors. We do note that this implementation reproduces the results in the
    original paper with minor deviations.
    """

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(40, (1, 13),
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(40, (Chans, 1), use_bias=False,
                          kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
    block1       = Activation(square)(block1)
    block1       = AveragePooling2D(pool_size=(1, 35), strides=(1, 7))(block1)
    block1       = Activation(log)(block1)
    block1       = Dropout(dropoutRate)(block1)
    flatten      = Flatten()(block1)
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)

    return Model(inputs=input_main, outputs=softmax)



In [12]:
checkpoint_path="weights.best.hdf5"
check_point = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_best_only=True,monitor='val_accuracy' ,save_weights_only=True, verbose=1)

In [13]:
num_classes =6
kernLength=10
model =EEGNet(nb_classes=num_classes, Chans = X_train.shape[1], Samples = X_train.shape[2],kernLength=10)
#model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',  metrics=['accuracy'])
optim=tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=optim,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,
                    epochs=50,
                    validation_data=(X_test,y_test),
                    batch_size=16,
                    verbose=1,callbacks=[check_point])

Epoch 1/50
17/17 [==============================] - ETA: 0s - loss: 1.8904 - accuracy: 0.1519
Epoch 1: val_accuracy improved from -inf to 0.23333, saving model to weights.best.hdf5
17/17 [==============================] - 43s 2s/step - loss: 1.8904 - accuracy: 0.1519 - val_loss: 1.7847 - val_accuracy: 0.2333
Epoch 2/50
17/17 [==============================] - ETA: 0s - loss: 1.6954 - accuracy: 0.3037
Epoch 2: val_accuracy did not improve from 0.23333
17/17 [==============================] - 40s 2s/step - loss: 1.6954 - accuracy: 0.3037 - val_loss: 1.7848 - val_accuracy: 0.1667
Epoch 3/50
17/17 [==============================] - ETA: 0s - loss: 1.5836 - accuracy: 0.3815
Epoch 3: val_accuracy did not improve from 0.23333
17/17 [==============================] - 29s 2s/step - loss: 1.5836 - accuracy: 0.3815 - val_loss: 1.7848 - val_accuracy: 0.2000
Epoch 4/50
17/17 [==============================] - ETA: 0s - loss: 1.5288 - accuracy: 0.3926
Epoch 4: val_accuracy did not improve from 0.233

KeyboardInterrupt: ignored

In [ ]:
 np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns

model.load_weights("/content/weights.best.hdf5")
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


# # Plot the confusion matrix
# cm = confusion_matrix(y_test, y_pred_classes)

# # Get class labels
# classes = unique_labels(y_true, y_pred)

# # Plot confusion matrix
# plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
# plt.colorbar()
# tick_marks = np.arange(len(classes))
# plt.xticks(tick_marks, classes)
# plt.yticks(tick_marks, classes)

# # Add labels to each cell
# thresh = cm.max() / 2.0
# for i in range(cm.shape[0]):
#     for j in range(cm.shape[1]):
#         plt.text(j, i, format(cm[i, j], 'd'),
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.title('Confusion Matrix')
# plt.show()

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
# GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
# from tensorflow.keras.backend import clear_session
# from tensorflow.keras.models import Model ,Sequential
# from tensorflow.keras.constraints import MaxNorm
# from keras.layers import Dense, Activation, Flatten
# from tensorflow.keras import regularizers

# def CNNModel():

#     clear_session()
#     model_m =Sequential()

#     model_m.add(Dropout(0.50, input_shape=(X_train.shape[1],X_train.shape[-1])))
#     model_m.add(Conv1D(128, 4, kernel_constraint = MaxNorm(1.),padding = 'same'))
#     model_m.add(Conv1D(128, 4, kernel_constraint = MaxNorm(1.),padding = 'same',))
#     model_m.add(BatchNormalization(axis = -1))
#     model_m.add(Activation("relu"))
#     model_m.add(MaxPool1D(2))
#     model_m.add(Dropout(0.50))
#     model_m.add(Conv1D(128, 10, kernel_constraint = MaxNorm(1.),padding = 'same'))
#     model_m.add(Conv1D(128, 10, kernel_constraint = MaxNorm(1.),padding = 'same'))
#     model_m.add(BatchNormalization(axis=-1))
#     model_m.add(Activation("relu"))
#     model_m.add(MaxPool1D(10))
#     model_m.add(Flatten())
#     model_m.add(Dropout(0.50))
#     model_m.add(Dense(64, kernel_regularizer = regularizers.l1_l2(l1=0, l2=1e-5), activation = 'relu', kernel_constraint = MaxNorm(1.)))
#     model_m.add(Dense(6, kernel_regularizer = regularizers.l1_l2(l1=0, l2=1e-5),activation = 'softmax',kernel_constraint = MaxNorm(1.)))

#     return model_m

# model.summary()

In [ ]:
# model=CNNModel()
# optim=tf.keras.optimizers.SGD(learning_rate=0.01) #SGD #reomve rest #shuffle  #classes one by one
# #model_m.compile(loss = "binary_crossentropy", optimizer = adam(lr = 0.001), metrics = ["acc"])
# model.compile(optimizer =optim,loss='categorical_crossentropy',metrics=['accuracy'])

# Epochs = 50

# history = model.fit(X_train,y_train,
#                     epochs=Epochs,
#                     validation_data=(X_test,y_test),
#                     batch_size=32,
#                     verbose=1)

In [ ]:
# history.history.keys()

In [ ]:
#doctor
# training_accuracy = history.history['accuracy']
# validation_accuracy = history.history['val_accuracy']

# training_loss = history.history['loss']
# validation_loss = history.history['val_loss']

# epochs_range=range(Epochs)

# plt.figure(figsize=(8, 8))
# plt.subplot(1, 2, 1)
# plt.plot(epochs_range, training_accuracy, label='Training Accuracy')
# plt.plot(epochs_range, validation_accuracy, label='Validation Accuracy')
# plt.legend(loc='lower right')
# plt.title('Training and Validation Accuracy')

# plt.subplot(1, 2, 2)
# plt.plot(epochs_range, training_loss, label='Training Loss')
# plt.plot(epochs_range, validation_loss, label='Validation Loss')
# plt.legend(loc='upper right')
# plt.title('Training and Validation Loss')
# plt.show()

In [ ]:
# model.evaluate(X_test,y_test)

In [ ]:
# p=model.predict(X_test)

In [ ]:
# np.argmax(p[5]) , np.argmax(y_test[5])


In [ ]:
# event_dict={
#  'elbow flexion': 1,
#  'elbow extension': 2,
#  'supination': 3,
#  'pronation': 4,
#  'hand close': 5,
#  'hand open': 6,
#  'rest': 7,
# }
# event_dict={
#  'Forward': 0,
#  'Backward ': 1,
#  'Left': 2,
#  'Right': 3,
#  'UP': 4,
#  'Down': 5,
#  'Rest': 6,
# }

# #d={1:0,2:1,3:2,4:3,5:4,6:5,7:6} previously used for preparing the labels

# def get_event_name(n):

#     for x,y in event_dict.items() :
#         if y == n :
#             return x

In [ ]:
# get_event_name(np.argmax(p[5])) ,get_event_name(np.argmax(y_test[5]))

In [ ]:
# plt.figure(figsize=(25,25))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
# #     plt.imshow(X_val[i])
#     color = 'green' if get_event_name(np.argmax(p[i])) == get_event_name(np.argmax(y_test[i])) else 'red'
#     plt.xlabel(get_event_name(np.argmax(p[i])),color=color)
# plt.show()